# Overview

The goal of this code is as follows:
1. From Wanikani, retrieve a list of all resources
2. Place those resources into a SQLite database

In [22]:
import os
import json
import sqlite3
import pandas as pd
import requests
import datetime
from pprint import pprint


token = 'YOUR TOKEN HERE'
endpoint = 'assignments'

The definition of wanikani_req is due to trunklayer. (See https://community.wanikani.com/t/python-api-call-headers-problem/45614/2)

In [9]:
def wanikani_req(token, endpoint):
    
    address = 'https://api.wanikani.com/v2'

    headers = {
        'Authorization': f'Bearer {token}'
    }

    with requests.get(f'{address}/{endpoint}', headers=headers) as r:
        response = r.json()
    #pprint(response)
    
    return response


Before I can pass the dates as date variables into the SQL database, I need to clean them up to meet SQL's datetime format

In [10]:
def date_cleanup(date):
    #The argument will take the form of something like '2021-09-29T00:27:27.473588Z'
    try:
        SQL_date = date[0:10] + ' ' + date[11:11+8]
    except:
        SQL_date = "NULL"
    
    return SQL_date

# Accessing the Wanikani API 

To begin, let's get the resources we need from wanikani

In [11]:
#ACQUIRING THE RESOURCES
#First, let's get a list of all the requests. Unfortunately, we are limited to 500 resources returned for collection endpoints
#This means if there's more than 500 resources we need to return, we need to navigate Wanikani's pagination system.
assignment_arr = []

req = wanikani_req(token, endpoint)
requests_to_get = req['total_count']

print("TOTAL REQUESTS TO GET: " + str(requests_to_get))

assignment_arr = assignment_arr + req['data']
requests_to_get = requests_to_get - len(req['data'])

print("After batch 1 added...")
print("TOTAL REQUESTS TO GET: " + str(requests_to_get))

i = 1
while requests_to_get > 0:
    i = i + 1

    #Now, access the info we need for the next page
    page_info = req['pages']['next_url'][40:40+24]

    #And modify the endpoint
    endpoint = 'assignments?' + page_info + 'started=true'

    #Now, make a request with the new endpoint
    req = wanikani_req(token, endpoint)
    assignment_arr = assignment_arr + req['data']
    requests_to_get = requests_to_get - len(req['data'])

    print("After batch " + str(i) + " added... ")
    print("TOTAL REQUESTS TO GET: " + str(requests_to_get))

#This final step is so that our data appears in a "JSON-like" format
assignment_dict = {"data": assignment_arr}
assignment_dict

TOTAL REQUESTS TO GET: 2258
After batch 1 added...
TOTAL REQUESTS TO GET: 1758
After batch 2 added... 
TOTAL REQUESTS TO GET: 1258
After batch 3 added... 
TOTAL REQUESTS TO GET: 758
After batch 4 added... 
TOTAL REQUESTS TO GET: 258
After batch 5 added... 
TOTAL REQUESTS TO GET: 0


{'data': [{'id': 261232563,
   'object': 'assignment',
   'url': 'https://api.wanikani.com/v2/assignments/261232563',
   'data_updated_at': '2022-04-28T14:56:44.721224Z',
   'data': {'created_at': '2021-09-29T00:27:27.476989Z',
    'subject_id': 19,
    'subject_type': 'radical',
    'srs_stage': 9,
    'unlocked_at': '2021-09-29T00:27:27.473588Z',
    'started_at': '2021-09-29T01:04:15.264089Z',
    'passed_at': '2021-10-03T01:03:12.740030Z',
    'burned_at': '2022-04-28T14:56:44.705779Z',
    'available_at': None,
    'resurrected_at': None,
    'hidden': False}},
  {'id': 261232564,
   'object': 'assignment',
   'url': 'https://api.wanikani.com/v2/assignments/261232564',
   'data_updated_at': '2022-04-26T20:40:33.176351Z',
   'data': {'created_at': '2021-09-29T00:27:27.483709Z',
    'subject_id': 6,
    'subject_type': 'radical',
    'srs_stage': 9,
    'unlocked_at': '2021-09-29T00:27:27.480595Z',
    'started_at': '2021-09-29T00:47:04.747634Z',
    'passed_at': '2021-10-03T01:02:2

Next, let's create the "schema", or structure for the SQL database that we'll use.

In [12]:
# If there's already a database file with the name we want, let's trash it first before doing anything:
db_path = 'wanikani.sqlite'

# Check if the file exists
if os.path.exists(db_path):
    # Delete the file
    os.remove(db_path)
    print(f'Database {db_path} deleted.')
else:
    print(f'Database {db_path} does not exist.')

# create an SQL schema based on the JSON-like data
schema = '''
CREATE TABLE assignment (
    id INTEGER PRIMARY KEY,
    subject_id INTEGER,
    subject_type TEXT,
    srs_stage INTEGER,
    unlocked_at DATETIME,
    started_at DATETIME,
    passed_at DATETIME,
    burned_at DATETIME,
    available_at DATETIME,
    resurrected_at DATETIME,
    hidden BOOL
);
'''

# create an SQLite database and connect to it
conn = sqlite3.connect('wanikani.sqlite')

# create the tables in the database
conn.execute(schema)
print('Schema successfully executed')
conn.close()

Database wanikani.sqlite deleted.
Schema successfully executed


Finally, let's build the database

In [13]:
conn = sqlite3.connect('wanikani.sqlite')
# insert the data into the database
for obj in assignment_dict['data']:
    #Note, the dates need to all be cleaned up before I can pass
    val_1 = obj['id']
    val_2 = obj['data']['subject_id']
    val_3 = obj['data']['subject_type']
    val_4 = obj['data']['srs_stage']
    val_5 = date_cleanup(obj['data']['unlocked_at'])
    val_6 = date_cleanup(obj['data']['started_at'])
    val_7 = date_cleanup(obj['data']['passed_at'])
    val_8 = date_cleanup(obj['data']['burned_at'])
    val_9 = date_cleanup(obj['data']['available_at'])
    val_10 = date_cleanup(obj['data']['resurrected_at'])
    val_11 = obj['data']['hidden']
    
    conn.execute("INSERT INTO assignment (id, subject_id, subject_type, srs_stage, unlocked_at, started_at, passed_at, burned_at, available_at, resurrected_at, hidden) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (val_1, val_2, val_3, val_4, val_5, val_6, val_7, val_8, val_9, val_10, val_11))

# commit the changes to the database
conn.commit()

# close the database connection
conn.close()

# Database Queries

We are now in a position to access and query the database. Below, I give some examples of database queries being performed:

In [20]:
days = 360

current_datetime = datetime.datetime.now()
cutoff_line = current_datetime - datetime.timedelta(days = days)

date_str = cutoff_line.strftime("%Y-%m-%d")
time_str = cutoff_line.strftime("%H:%M:%S")

cutoff_date =  date_str + " " + time_str
print("cutoff_date " + cutoff_date)

query = f"SELECT * FROM assignment WHERE started_at < '{cutoff_date}' AND burned_at = 'NULL' AND subject_type = 'radical'"

cutoff_date 2022-03-04 23:34:11


In [21]:
conn = sqlite3.connect('wanikani.sqlite')

query_table = pd.read_sql_query(query, conn)

pprint(query_table)
# cur = conn.cursor()
# cur.execute('SELECT * FROM assignment')
# results = cur.fetchall()
# print(type(results))
# for row in results:
#     pprint(row)
    
conn.close()

          id  subject_id subject_type  srs_stage          unlocked_at  \
0  262719708          54      radical          8  2021-10-09 03:52:10   
1  262719712          37      radical          8  2021-10-09 03:52:10   
2  268851559          62      radical          6  2021-11-22 00:58:55   
3  268851560          64      radical          8  2021-11-22 00:58:55   
4  268851562          61      radical          8  2021-11-22 00:58:55   
5  278398921          78      radical          8  2022-01-31 04:13:03   
6  278398922          76      radical          7  2022-01-31 04:13:03   
7  278398924          81      radical          8  2022-01-31 04:13:03   

            started_at            passed_at burned_at         available_at  \
0  2021-10-13 04:22:19  2021-10-24 21:44:01      NULL  2023-04-20 00:00:00   
1  2021-10-10 02:53:34  2021-10-17 01:44:35      NULL  2023-04-13 04:00:00   
2  2021-11-28 21:39:00  2022-01-15 18:08:55      NULL  2023-03-12 21:00:00   
3  2021-11-28 21:38:57  2022-0